# Examples of all decoders (except Kalman Filter)

In this example notebook, we:
1. Import the necessary packages
2. Load a data file (spike trains and outputs we are predicting)
3. Preprocess the data for use in all decoders
4. Run all decoders and print the goodness of fit
5. Plot example decoded outputs

See "Examples_kf_decoder" for a Kalman filter example. <br>
Because the Kalman filter utilizes different preprocessing, we don't include an example here. to keep this notebook more understandable

## 1. Import Packages

Below, we import both standard packages, and functions from the accompanying .py files

In [ ]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

#Import function to get the covariate matrix that includes spike history from previous bins
from preprocessing_funcs import get_spikes_with_history

#Import metrics
from metrics import get_R2
from metrics import get_rho

#Import decoder functions
from decoders import WienerCascadeDecoder
from decoders import WienerFilterDecoder
from decoders import DenseNNDecoder
from decoders import SimpleRNNDecoder
from decoders import GRUDecoder
from decoders import LSTMDecoder
from decoders import XGBoostDecoder
from decoders import SVRDecoder

## 2. Load Data
The data file for this example can be downloaded at https://dl.dropboxusercontent.com/u/2944301/Decoding_Data/example_data_s1.pickle. It was recorded by Raeed Chowdhury from Lee Miller's lab at Northwestern.


The data that we load is in the format described below. We have another example notebook, "Example_format_data", that may be helpful towards putting the data in this format.

Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin

The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"

 

In [ ]:
folder='/Users/guitchounts/Dropbox (coxlab)/Ephys/Data/GRat30/636380500192558468' #ENTER THE FOLDER THAT YOUR DATA IS IN
# folder='/home/jglaser/Data/DecData/' 
# folder='/Users/jig289/Dropbox/Public/Decoding_Data/'

with open(folder+'/jerk_10hz.pickle','rb') as f:
#     neural_data,vels_binned=pickle.load(f,encoding='latin1') #If using python 3
    jerk=pickle.load(f) #If using python 2

In [ ]:
jerk = jerk.T

In [1]:
jerk.shape

NameError: name 'jerk' is not defined

In [ ]:
import h5py

In [ ]:
f = h5py.File(folder+'/lfp_spec_win1step01.mat','r')
freqs = f['f'][:]
time = f['t'][:]
lfp_spec = f['lfp_spec'][:]

In [ ]:
f.close()

In [ ]:
lfp_spec.shape

In [ ]:
def get_freq_idx(freqs,desired_freq): # make desired_freq a tuple, e.g. (0,4)
    idx = []
    for counter,value in enumerate(freqs):
        if  desired_freq[0] <= value <= desired_freq[1]:
            #yield counter
            idx.append(counter)
    return idx


def get_power(spec,freq_range,freqs):

   

    idx = get_freq_idx(freqs,freq_range)
        #idx_0_4 = get_freq_idx(freqs,(0,4))
        #idx_5_30 = get_freq_idx(freqs,(5,30))
        #idx_30_100 = get_freq_idx(freqs,(30,100))

    power = np.mean(spec[idx,:],0)
    #print 'shape of Pxx[idx,:] = ', Pxx[idx,:].shape
    #power = np.trapz(spec[idx,:],axis=0)
        #power_0_4 = np.mean(Pxx[idx_0_4,:],0)
        #power_5_30 = np.mean(Pxx[idx_5_30,:],0) # take mean on 0-th dim of Pxx -> this gives power over time. 
        #power_30_100 = np.mean(Pxx[idx_30_100,:],0)


    #print 'shape of power = ', power.shape

    return power

In [ ]:
####### get LFP power for each channel in the four bands:

lfp_power = np.zeros([64*4,time.shape[0]])  ## 64 channels x 4 bands

for ch in range(64):
    power_0_4 = get_power(lfp_spec[ch,:,:],[0,4],freqs)
    power_5_15 = get_power(lfp_spec[ch,:,:],[5,15],freqs)
    power_15_40 = get_power(lfp_spec[ch,:,:],[15,40],freqs)
    power_40_100 = get_power(lfp_spec[ch,:,:],[40,100],freqs)
    lfp_power[ch*4:(ch+1)*4,:] = power_0_4,power_5_15,power_15_40,power_40_100

In [ ]:
neural_data = lfp_power.T

In [ ]:
neural_data.shape

In [ ]:
jerk_power = np.mean(jerk_spec,axis=1).T

In [ ]:
jerk_power.shape

In [ ]:
plt.plot(jerk_power[:,0])

In [ ]:
for ch in range(64):
    power_0_4 = get_power(lfp_spec[ch,:,:],[0,4],freqs)
    power_5_15 = get_power(lfp_spec[ch,:,:],[5,15],freqs)
    power_15_40 = get_power(lfp_spec[ch,:,:],[15,40],freqs)
    power_40_100 = get_power(lfp_spec[ch,:,:],[40,100],freqs)

In [ ]:
f, axarr = plt.subplots(3, sharex=True,dpi=600)
axarr[0].plot(spike_time_vec,neural_data[:,0],linewidth=.25)
axarr[1].plot(jerk_time,jerk_power[:,0],linewidth=.25)
axarr[2].plot(spike_time_vec,raw_jerk[:,0],linewidth=.25)



In [ ]:
lfp_spec.shape

In [ ]:
plt.scatter(np.mean(lfp_spec,axis=(0,1)),jerk[:,0],alpha=.1,marker='o')
#plt.ylim([-.002,.002])

In [ ]:
neural_data

## 3. Preprocess Data

### 3A. User Inputs
The user can define what time period to use spikes from (with respect to the output).

In [ ]:
bins_before=15 #How many bins of neural data prior to the output are used for decoding
bins_current=1 #Whether to use concurrent time bin of neural data
bins_after=15 #How many bins of neural data after the output are used for decoding

### 3B. Format Covariates

#### Format Input Covariates

In [ ]:
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
# Function to get the covariate matrix that includes spike history from previous bins
X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
#Put in "flat" format, so each "neuron / time" is a single feature
X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

In [ ]:
X.shape

In [ ]:
X_flat.shape

#### Format Output Covariates

In [ ]:
#Set decoding output
#y=jerk_power
y=jerk

In [ ]:
y.shape

### 3C. Split into training / testing / validation sets
Note that hyperparameters should be determined using a separate validation set. 
Then, the goodness of fit should be be tested on a testing set (separate from the training and validation sets).

#### User Options

In [ ]:
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.5]
testing_range=[0.7, 0.85]
valid_range=[0.5,1]

#### Split Data

In [ ]:
num_examples=X.shape[0]

#Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
#This makes it so that the different sets don't include overlapping neural data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples))+bins_before,np.int(np.round(training_range[1]*num_examples))-bins_after)
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples))+bins_before,np.int(np.round(testing_range[1]*num_examples))-bins_after)
valid_set=np.arange(np.int(np.round(valid_range[0]*num_examples))+bins_before,np.int(np.round(valid_range[1]*num_examples))-bins_after)

#Get training data
X_train=X[training_set,:,:]
X_flat_train=X_flat[training_set,:]


y_train=y[training_set,:]

#Get testing data
X_test=X[testing_set,:,:]
X_flat_test=X_flat[testing_set,:]

y_test=y[testing_set,:]

#Get validation data
X_valid=X[valid_set,:,:]
X_flat_valid=X_flat[valid_set,:]


y_valid=y[valid_set,:]

### 3D. Process Covariates
We normalize (z_score) the inputs and zero-center the outputs.
Parameters for z-scoring (mean/std.) should be determined on the training set only, and then these z-scoring parameters are also used on the testing and validation sets.

In [ ]:
#Z-score "X" inputs. 
X_train_mean=np.nanmean(X_train,axis=0)
X_train_std=np.nanstd(X_train,axis=0)
X_train=(X_train-X_train_mean)/X_train_std
X_test=(X_test-X_train_mean)/X_train_std
X_valid=(X_valid-X_train_mean)/X_train_std


#Z-score "X_flat" inputs. 
X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
X_flat_train_std=np.nanstd(X_flat_train,axis=0)
X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
X_flat_valid=(X_flat_valid-X_flat_train_mean)/X_flat_train_std

#Zero-center outputs
y_train_mean=np.mean(y_train,axis=0)
y_train=y_train-y_train_mean
y_test=y_test-y_train_mean
y_valid=y_valid-y_train_mean

In [ ]:
f, axarr = plt.subplots(2, sharex=True,dpi=600)
axarr[0].plot(X_flat_train[0:1000,100],linewidth=.25)
axarr[1].plot(y_train[0:1000,0],linewidth=.25)
#axarr[2].plot(spike_time_vec,raw_jerk[:,0],linewidth=.25)



## 4. Run Decoders
Note that in this example, we are evaluating the model fit on the validation set

### 4A. Wiener Filter (Linear Regression)

In [ ]:
from sklearn import linear_model

In [ ]:
def ridgeCV_model(train_x,train_y,test_x,test_y):
    
    model = linear_model.RidgeCV(alphas=[0.1,1.0,10.],normalize=True,fit_intercept=True)
    model.fit(train_x,train_y)
    prediction = model.predict(test_x)
    score = model.score(test_x,test_y)
    print 'Model score R^2 = ', score
    plt.scatter(test_y,prediction,alpha=0.1,marker='o')
    plt.axis('equal')
    return model
    

In [ ]:
y_test.shape

In [ ]:
X_flat_test.shape

In [ ]:
X_flat_train.shape

In [ ]:
ridge_model = ridgeCV_model(X_flat_train,y_train,X_flat_test,y_test)

In [ ]:
#Declare model
model_wf=WienerFilterDecoder()

#Fit model
model_wf.fit(X_flat_train,y_train)

#Get predictions
y_valid_predicted_wf=model_wf.predict(X_flat_valid)

#Get metric of fit
R2s_wf=get_R2(y_valid,y_valid_predicted_wf)
print('R2s:', R2s_wf)

In [ ]:
f, axarr = plt.subplots(2, sharex=True,dpi=600)
axarr[0].plot(y_valid,linewidth=0.1)

axarr[1].plot(y_valid_predicted_wf,linewidth=0.1,color='red')


In [ ]:
plt.scatter(y_valid,y_valid_predicted_wf,alpha=0.1,marker='o')
plt.axis('equal')

### 4B. Wiener Cascade (Linear Nonlinear Model)

In [ ]:
#Declare model
model_wc=WienerCascadeDecoder(degree=3)

#Fit model
model_wc.fit(X_flat_train,y_train)

#Get predictions
y_valid_predicted_wc=model_wc.predict(X_flat_valid)

#Get metric of fit
R2s_wc=get_R2(y_valid,y_valid_predicted_wc)
print('R2s:', R2s_wc)

### 4C. XGBoost (Extreme Gradient Boosting)

In [ ]:
#Declare model
model_xgb=XGBoostDecoder(max_depth=3,num_round=200,eta=0.3,gpu=-1) 

#Fit model
model_xgb.fit(X_flat_train, y_train)

#Get predictions
y_valid_predicted_xgb=model_xgb.predict(X_flat_valid)

#Get metric of fit
R2s_xgb=get_R2(y_valid,y_valid_predicted_xgb)
print('R2s:', R2s_xgb)

### 4D. SVR (Support Vector Regression)

In [ ]:
#The SVR works much better when the y values are normalized, so we first z-score the y values
#They have previously been zero-centered, so we will just divide by the stdev (of the training set)
y_train_std=np.nanstd(y_train,axis=0)
y_zscore_train=y_train/y_train_std
y_zscore_test=y_test/y_train_std
y_zscore_valid=y_valid/y_train_std

#Declare model
model_svr=SVRDecoder(C=5, max_iter=4000)

#Fit model
model_svr.fit(X_flat_train,y_zscore_train)

#Get predictions
y_zscore_valid_predicted_svr=model_svr.predict(X_flat_valid)

#Get metric of fit
R2s_svr=get_R2(y_zscore_valid,y_zscore_valid_predicted_svr)
print('R2s:', R2s_svr)

### 4E. Dense Neural Network

In [ ]:
#Declare model
model_dnn=DenseNNDecoder(units=400,dropout=0.25,num_epochs=10)

#Fit model
model_dnn.fit(X_flat_train,y_train)

#Get predictions
y_valid_predicted_dnn=model_dnn.predict(X_flat_valid)

#Get metric of fit
R2s_dnn=get_R2(y_valid,y_valid_predicted_dnn)
print('R2s:', R2s_dnn)

### 4F. Simple RNN

In [ ]:
#Declare model
model_rnn=SimpleRNNDecoder(units=400,dropout=0,num_epochs=5)

#Fit model
model_rnn.fit(X_train,y_train)

#Get predictions
y_valid_predicted_rnn=model_rnn.predict(X_valid)

#Get metric of fit
R2s_rnn=get_R2(y_valid,y_valid_predicted_rnn)
print('R2s:', R2s_rnn)

### 4G. GRU (Gated Recurrent Unit)

In [ ]:
#Declare model
model_gru=GRUDecoder(units=400,dropout=0,num_epochs=5)

#Fit model
model_gru.fit(X_train,y_train)

#Get predictions
y_valid_predicted_gru=model_gru.predict(X_valid)

#Get metric of fit
R2s_gru=get_R2(y_valid,y_valid_predicted_gru)
print('R2s:', R2s_gru)

### 4H. LSTM (Long Short Term Memory)

In [ ]:
#Declare model
model_lstm=LSTMDecoder(units=400,dropout=0,num_epochs=5)

#Fit model
model_lstm.fit(X_train,y_train)

#Get predictions
y_valid_predicted_lstm=model_lstm.predict(X_valid)

#Get metric of fit
R2s_lstm=get_R2(y_valid,y_valid_predicted_lstm)
print('R2s:', R2s_lstm)

## 5. Make Plots

In [ ]:
#As an example, I plot an example 1000 values of the x velocity (column index 0), both true and predicted with the Wiener filter
#Note that I add back in the mean value, so that both true and predicted values are in the original coordinates
fig_x_wf=plt.figure()
plt.plot(y_valid[1000:2000,0]+y_train_mean[0],'b')
plt.plot(y_valid_predicted_wf[1000:2000,0]+y_train_mean[0],'r')

#Save figure
# fig_x_wf.savefig('x_velocity_decoding.eps')